In [ ]:
import re

def clean_product_name_final(product_name):
    """제품명에서 모든 부가 설명을 제거하고 핵심 제품명만 추출합니다."""
    name = product_name

    # 1. 괄호와 대괄호 안의 내용 전체 제거: 기획/증정/세트 정보 삭제
    name = re.sub(r'\[.*?\]', '', name).strip()
    name = re.sub(r'\(.*?\)', '', name).strip()

    # 2. 용량 및 수량 정보 제거: 100ml, 50g, 2ea 등 숫자+단위 제거
    # 숫자가 포함된 용량 단위 패턴을 제거합니다. (100ml, 50g 등)
    name = re.sub(r'\s*\d+[\s\d]*[mMgG][lLgG]|[\s\d]*[eE][aA]', '', name, flags=re.IGNORECASE).strip()

    # 3. 핵심 마케팅 키워드 제거
    # 자주 등장하는 마케팅/부가 설명 키워드를 제거합니다.
    keywords_to_remove = ['기획', '증정', '대용량', '더블', '세트', '단품', '올영픽', '리즈PICK', '수분광', '수분천재크림', '점보']
    for keyword in keywords_to_remove:
        # 단어 경계(\b)로 정확하게 키워드만 제거
        name = re.sub(r'\b' + re.escape(keyword) + r'\b', '', name, flags=re.IGNORECASE).strip()

    # 4. 연속된 공백을 하나로 줄이고 최종 정리
    name = re.sub(r'\s+', ' ', name).strip()
    
    return name if name else product_name

# 예시: 라로슈포제 시카플라스트 멀티 리페어 크림

1. 정리 후: 라로슈포제 시카플라스트 멀티 리페어 크림
2. 정리 후: 에스네이처 아쿠아 스쿠알란 수분크림


In [ ]:
def clean_and_split_ingredients(ingredients_list, api_ingredient_names):
    """
    성분 리스트를 분해하고 불순물을 제거하며, 공공 API 명칭을 기준으로 검증합니다.
    
    :param ingredients_list: 스크래핑된 원본 성분 리스트 (예: ["정제수 다이카프릴릴에터", "스쿠알란(150,000ppm)"])
    :param api_ingredient_names: 공공 API에서 추출한 표준 성분명 리스트 (Set 형태 권장)
    :return: 정제된 성분 리스트
    """
    
    # 공공 API에서 가져온 성분명 리스트를 Set으로 변환하여 빠른 검색에 사용
    api_set = set(api_ingredient_names)
    
    # 1. 단일 텍스트로 결합 및 불순물 제거 준비
    # 리스트를 하나의 문자열로 결합하고, 불필요한 공백이나 개행문자를 제거합니다.
    raw_text = ' '.join(ingredients_list).replace('\n', ' ').strip()
    
    # ppm 등 농도 정보와 괄호 내부의 내용 제거 (성분명 분리에 방해되는 요소)
    raw_text = re.sub(r'\([^)]*\)', ' ', raw_text).strip()
    # 대괄호 안의 내용도 제거
    raw_text = re.sub(r'\[.*?\]', ' ', raw_text).strip()
    
    # 제품명/용량과 성분명이 붙어있는 경우를 대비해 자주 보이는 구분자(숫자, 용량단위 등)를 공백으로 치환
    raw_text = re.sub(r'\d+[\s\d]*[mMgG][lLgG]|\d+[\s\d]*[eE][aA]', ' ', raw_text, flags=re.IGNORECASE)
    
    # 2. 성분명으로 텍스트 재분리 (핵심 로직)
    tokenized_ingredients = set()
    current_text = raw_text
    
    # API 성분명과 매칭하여 성분 추출: 긴 성분명이 먼저 매칭되도록 길이 순으로 정렬
    # '소듐하이알루로네이트'가 '소듐'보다 먼저 추출되어야 정확합니다.
    sorted_api_list = sorted(list(api_set), key=len, reverse=True)

    for api_name in sorted_api_list:
        # 단어 경계(\b)를 사용하여 정확하게 성분명과 일치하는지 확인
        if re.search(r'\b' + re.escape(api_name) + r'\b', current_text):
            tokenized_ingredients.add(api_name)
            # 매칭된 성분명을 공백으로 치환하여 중복 매칭과 잔여 텍스트 오류를 방지
            current_text = re.sub(r'\b' + re.escape(api_name) + r'\b', ' ', current_text)

    # 3. 최종 불순물 제거 및 리스트 정리
    cleaned_list = []
    for item in tokenized_ingredients:
        item = item.strip()
        
        # 숫자로만 이루어진 잔여물이나 너무 짧은 단어(3자 이하) 제거
        if re.fullmatch(r'^\d+[\s\d]*$', item) or len(item) <= 3:
            continue
        
        # 최종적으로 API 표준 목록에 포함된 성분만 인정하여 중복 검증
        if item in api_set:
            cleaned_list.append(item)
            
    # 최종 중복 제거 및 정렬
    return sorted(list(set(cleaned_list)))

In [ ]:
import pandas as pd
import numpy as np
import re # 정규 표현식 라이브러리

# --- 설정 (파일 이름은 실제 파일명에 맞게 수정하세요) ---
COOS_FILE = "coos_ingredient_database_final.csv"
OLIVE_YOUNG_FILE = "oliveyoung_products.csv" # ⚠️ 실제 올리브영 파일명으로 수정 필요
OUTPUT_FILE = "integrated_cosmetic_data.csv"

# ----------------------------------------------------------------------
# 0. 데이터 불러오기
# ----------------------------------------------------------------------
try:
    # COOS 성분 데이터 (스크래핑 결과)
    df_coos = pd.read_csv(COOS_FILE)
    print(f"COOS 데이터 로드 완료: {len(df_coos)}개 성분")
except FileNotFoundError:
    print(f"❌ 오류: COOS 파일 '{COOS_FILE}'을 찾을 수 없습니다. 파일명을 확인해 주세요.")
    exit()

try:
    # 올리브영 상품 데이터 (이전에 스크래핑했던 결과)
    # ⚠️ 이 파일은 '상품명', '가격', '성분_목록' (콤마로 구분된 성분 리스트) 컬럼이 있다고 가정합니다.
    df_oly = pd.read_csv(OLIVE_YOUNG_FILE)
    print(f"올리브영 데이터 로드 완료: {len(df_oly)}개 상품")
except FileNotFoundError:
    # 테스트를 위해 올리브영 데이터가 없을 경우 가상 데이터 생성
    print(f"⚠️ 올리브영 파일 '{OLIVE_YOUNG_FILE}'이 없어 가상 데이터를 생성합니다.")
    df_oly = pd.DataFrame({
        '상품ID': [101, 102, 103],
        '상품명': ['A사 수딩 크림', 'B사 톤업 선크림', 'C사 샴푸'],
        '브랜드': ['A사', 'B사', 'C사'],
        '가격': [25000, 32000, 18000],
        # COOS 데이터와 매칭 테스트를 위한 성분 목록
        '성분_목록': [
            "정제수, 글리세린, 히비스커스꽃추출물, 히스티딘, 토코페롤",
            "정제수, 징크옥사이드, 티타늄디옥사이드, 흰버드나무껍질추출물, 나이아신아마이드",
            "라우릴글루코사이드, 코카미도프로필베타인, 정제수"
        ]
    })


# ----------------------------------------------------------------------
# 1단계: COOS 데이터 클리닝 및 표준화
# ----------------------------------------------------------------------
print("\n[1단계] COOS 성분 데이터 정리 시작...")

# 1. 태그 목록 정리 (불필요한 노이즈 제거)
# 스크래핑 코드에서 이미 필터링했으나, 혹시 모를 잔여 노이즈를 정리합니다.
def clean_tags(tags):
    if not isinstance(tags, str):
        return ""
    
    # 콤마로 분리 후 불필요한 패턴 제거
    tags_list = [t.strip() for t in tags.split(',')]
    
    # 길이가 짧거나, 특정 코드가 포함된 태그를 다시 제거 (예: AI, KO, EN, 식, 가능, 불가)
    NOISE_PATTERNS = re.compile(r'^(AI|KO|EN|JP|EU|식|가능|불가|EWG)$', re.IGNORECASE)
    
    cleaned_tags = [
        t for t in tags_list 
        if t and len(t) > 2 and not NOISE_PATTERNS.match(t)
    ]
    return ', '.join(cleaned_tags)

df_coos['태그_목록_정리'] = df_coos['태그_목록'].apply(clean_tags)

# 2. 표준 성분명 컬럼 생성 (조인을 위한 키)
# 원료명, 영문명 모두 공백, 괄호 등을 제거하여 조인 시 오류를 줄일 수 있습니다.
def create_standard_name(name):
    if not isinstance(name, str):
        return None
    # 띄어쓰기, 괄호 제거 후 소문자 변환
    return re.sub(r'[\s\(\)\[\]]+', '', name).lower()

df_coos['표준성분명'] = df_coos['원료명'].apply(create_standard_name)

print("COOS 데이터 정리 및 표준 성분 키 생성 완료.")


# ----------------------------------------------------------------------
# 2단계: 올리브영 데이터 성분 목록 분리 (Explode)
# ----------------------------------------------------------------------
print("\n[2단계] 올리브영 성분 목록 분리 시작...")

# 1. '성분_목록' 컬럼을 콤마(,) 기준으로 리스트로 변환
df_oly['성분_리스트'] = df_oly['성분_목록'].str.split(r',(?![^\(]*\))') 
# (팁: 괄호 안의 콤마는 분리하지 않는 정규표현식, 복잡한 성분명 처리)

# 2. explode (데이터 폭발): 하나의 상품을 여러 성분 행으로 분리
df_oly_exploded = df_oly.explode('성분_리스트')

# 3. 분리된 성분명 클리닝 및 표준 성분 키 생성
df_oly_exploded['원료명'] = df_oly_exploded['성분_리스트'].str.strip()
df_oly_exploded['표준성분명'] = df_oly_exploded['원료명'].apply(create_standard_name)

# 불필요한 행(빈 성분명) 제거
df_oly_exploded.dropna(subset=['표준성분명'], inplace=True)

print(f"올리브영 데이터 폭발 완료. 총 {len(df_oly_exploded)}개의 성분 항목 생성.")


# ----------------------------------------------------------------------
# 3단계: 표준 성분 매핑 및 데이터 통합 (JOIN)
# ----------------------------------------------------------------------
print("\n[3단계] 두 데이터셋 통합(Merge) 시작...")

# COOS 데이터에서 필요한 컬럼만 선택 (중복 조인을 막기 위해)
coos_cols = ['표준성분명', '원료명', '영문명', 'CAS_No', '설명_요약', '태그_목록_정리', '상세_URL']
df_coos_clean = df_coos[coos_cols].rename(columns={'원료명': 'COOS_원료명'}).drop_duplicates(subset=['표준성분명'])


# 올리브영 상품 데이터와 COOS 성분 데이터를 '표준성분명'을 기준으로 Left Join
# Left Join을 사용하면, COOS 데이터에 없는 성분이라도 올리브영 상품 정보는 유지됩니다.
df_final = pd.merge(
    df_oly_exploded, 
    df_coos_clean, 
    on='표준성분명', 
    how='left', 
    suffixes=('_OLY', '_COOS')
)

# ----------------------------------------------------------------------
# 4. 최종 정리 및 저장
# ----------------------------------------------------------------------

# 최종 컬럼 순서 지정 및 정리
df_final = df_final.rename(columns={'원료명_OLY': '사용_원료명_OLY'})

final_columns = [
    '상품명', '브랜드', '가격', # 상품 정보
    '사용_원료명_OLY', 'COOS_원료명', # 매핑된 성분명
    '영문명', 'CAS_No', '설명_요약', '태그_목록_정리', # COOS 성분 속성
    '상세_URL' # COOS 링크
]

df_final = df_final[[col for col in final_columns if col in df_final.columns]]

# 매핑되지 않은 성분에 대한 처리 (COOS 정보가 없는 경우)
df_final['COOS_매칭여부'] = np.where(df_final['CAS_No'].isnull(), '미매칭', '매칭완료')

df_final.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')

print("\n==================================================")
print(f"✅ 데이터 통합 및 표준화 완료!")
print(f"최종 통합 데이터셋 크기: {len(df_final)} 행")
print(f"결과 파일: {OUTPUT_FILE}")
print("==================================================")


In [3]:
import pandas as pd
import numpy as np
import re
import os
import warnings
from itertools import combinations 
warnings.filterwarnings('ignore', category=FutureWarning)

# ----------------------------------------------------------------------
# --- 설정 ---
# ----------------------------------------------------------------------
# COOS 데이터베이스 파일
COOS_FILE = "coos_ingredient_database.csv"

# 올리브영 JSON 파일 목록
OLIVE_YOUNG_FILES = [
    "oliveyoung_로션_raw_limited.json",
    "oliveyoung_미스트_오일_raw_limited.json",
    "oliveyoung_스킨_토너_raw_limited.json",
    "oliveyoung_에센스_세럼_앰플_raw_limited.json",
    "oliveyoung_크림_raw_limited.json",
]
OUTPUT_FILE = "integrated_product_ingredient_normalized.csv"

# ----------------------------------------------------------------------
# --- 데이터 클리닝 유틸리티 (Normalization Functions) ---
# data_cleaner_utils.py 파일의 내용을 그대로 포함합니다.
# ----------------------------------------------------------------------

def clean_tags(tags):
    """
    COOS 데이터의 태그 목록에서 불필요한 노이즈를 제거하고 정리합니다.
    """
    if not isinstance(tags, str): return ""
    tags_list = [t.strip() for t in tags.split(',')]
    # 불필요한 패턴(AI, KO, JP 등) 및 짧은 태그 제거를 위한 정규식
    NOISE_PATTERNS = re.compile(r'^(AI|KO|EN|JP|EU|식|가능|불가|EWG|EWG-Green)$', re.IGNORECASE)
    cleaned_tags = [
        t for t in tags_list 
        if t and len(t) > 2 and not NOISE_PATTERNS.match(t)
    ]
    return ', '.join(cleaned_tags)

def create_standard_name(name):
    """
    성분명에서 특수문자 등을 제거하고 소문자화하여 표준 매핑 키(Standard Key)를 만듭니다.
    """
    if not isinstance(name, str): return None
    # 문자, 숫자, 한글, 영어 외의 모든 문자를 공백으로 치환 후 제거
    cleaned_name = re.sub(r'[^\w가-힣]+', '', name).lower()
    return cleaned_name

# ----------------------------------------------------------------------
# --- 데이터 로딩 및 통합 함수 ---
# ----------------------------------------------------------------------

def load_and_consolidate_oliveyoung_data(file_list):
    """여러 올리브영 JSON 파일을 통합하고 카테고리 정보를 추가합니다."""
    all_dfs = []
    print("\n[0.1단계] 올리브영 JSON 파일 통합 시작...")
    for file_path in file_list:
        try:
            base_name = os.path.basename(file_path)
            # 파일명에서 카테고리 추출
            category_part = base_name.replace('oliveyoung_', '').replace('_raw_limited.json', '')
            category_name = category_part.replace('_', '/')

            df = pd.read_json(file_path, encoding='utf-8')
            df['카테고리'] = category_name
            all_dfs.append(df)
            
        except FileNotFoundError:
            print(f"   ❌ 경고: 파일 '{file_path}'을 찾을 수 없습니다. 건너뜁니다.")
        except Exception as e:
            print(f"   ❌ 경고: 파일 '{file_path}' 로드 중 오류 발생: {e}. 건너뜁니다.")
            
    if not all_dfs:
        print("   ❌ 오류: 로드된 올리브영 데이터가 없습니다. 스크립트를 종료합니다.")
        return None
        
    df_consolidated = pd.concat(all_dfs, ignore_index=True)
    if '상품ID' not in df_consolidated.columns:
        df_consolidated['상품ID'] = df_consolidated.index.astype(str)
        
    print(f"✅ 총 {len(df_consolidated)}개 상품 통합 완료.")
    return df_consolidated

def clean_and_standardize_coos(df_coos):
    """COOS 데이터를 클리닝하고 표준 매핑 키를 생성합니다."""
    print("\n[1단계] COOS 성분 데이터 정리 및 표준화 시작...")
    
    # 1. 태그 정리 및 표준 성분명(매핑 키) 생성
    df_coos['태그_목록_정리'] = df_coos['태그_목록'].apply(clean_tags)
    df_coos['표준성분명'] = df_coos['원료명'].apply(create_standard_name)
    
    # 2. 모델에 필요한 속성 컬럼 정의
    coos_cols = ['표준성분명', '원료명', '영문명', 'CAS_No', '설명_요약', '태그_목록_정리', '상세_URL']
    df_coos_clean = df_coos[coos_cols].rename(columns={'원료명': 'COOS_원료명'}).drop_duplicates(subset=['표준성분명'], keep='first')
    
    return df_coos_clean

def verify_and_rename_ingredient_column(df, expected_col='성분_목록'):
    """
    df에 필수 성분 컬럼이 있는지 확인하고, 없으면 대안 컬럼을 찾아 이름을 변경합니다.
    """
    if expected_col in df.columns:
        return df

    # 스크래핑 데이터에서 흔히 사용되는 대안 이름 목록 (성분리스트 추가)
    ALTERNATIVE_NAMES = ['성분리스트', 'ingredients', '전성분', 'Ingredient_List', '성분목록']
    
    found_alt = None
    for alt in ALTERNATIVE_NAMES:
        if alt in df.columns:
            df.rename(columns={alt: expected_col}, inplace=True)
            print(f"   ⚠️ 경고: '{expected_col}' 컬럼 대신 '{alt}' 컬럼을 사용하여 이름을 변경했습니다.")
            found_alt = True
            break
    
    if not found_alt:
        print(f"   ❌ 치명적 오류: 통합된 올리브영 데이터에 필수 컬럼 '{expected_col}' 또는 그 대안이 없습니다.")
        print(f"   현재 컬럼 목록: {df.columns.tolist()}")
        # KeyError를 다시 발생시켜 메인 블록에서 처리하도록 함
        raise KeyError(expected_col) 

    return df

def verify_and_rename_metadata_columns(df):
    """
    상품 메타데이터 컬럼 중 존재하는 컬럼(제품명)만 확인하고 표준화합니다.
    (브랜드, 가격, 리뷰수, 평점은 데이터에 없으므로 체크하지 않음)
    """
    # {표준이름: [대안 이름 목록]}
    REQUIRED_MAPPING = {
        # 사용자 피드백에 따라, 존재하는 '제품명'만 '상품명'으로 표준화합니다.
        '상품명': ['제품명', 'name', 'product_name'],
    }
    
    current_cols = df.columns.tolist()
    
    print("   [1.6단계] 메타데이터 컬럼 표준화 시작...")

    # 필수 컬럼 검사 및 이름 변경
    for expected, alternatives in REQUIRED_MAPPING.items():
        if expected not in current_cols:
            for alt in alternatives:
                if alt in current_cols:
                    df.rename(columns={alt: expected}, inplace=True)
                    print(f"      - '{alt}'을(를) '{expected}'으로(로) 변경했습니다.")
                    break
    
    # 누락된 컬럼(브랜드, 가격, 리뷰수, 평점)에 대한 경고는 생략합니다.
    print("   ✅ 메타데이터 표준화 완료 (상품명만 존재 확인).")

    return df

# ----------------------------------------------------------------------
# --- 메인 실행 로직 ---
# ----------------------------------------------------------------------
def main():
    
    # 0. 데이터 로드
    df_coos = None # NameError 방지를 위해 초기화
    try:
        df_coos = pd.read_csv(COOS_FILE)
        print(f"\n[0.2단계] COOS 데이터 로드 완료: {len(df_coos)}개 성분")
    except FileNotFoundError:
        print(f"\n❌ 오류: COOS 파일 '{COOS_FILE}'을 찾을 수 없습니다. 파일명을 확인해 주세요.")
        return # main() 함수 내부에서는 return 사용 가능
    except Exception as e:
        print(f"\n❌ 오류: COOS 파일 로드 중 예상치 못한 오류 발생: {e}")
        return # main() 함수 내부에서는 return 사용 가능

    df_oly = load_and_consolidate_oliveyoung_data(OLIVE_YOUNG_FILES)
    if df_oly is None: return # main() 함수 내부에서는 return 사용 가능

    # 1.5단계: 필수 성분 컬럼 확인 및 이름 변경 (성분 목록 키 표준화)
    try:
        df_oly = verify_and_rename_ingredient_column(df_oly, expected_col='성분_목록')
    except KeyError as e:
        print("\n   ⚠️ 데이터 오류로 인해 스크립트를 종료합니다. JSON 파일의 성분 목록 키를 확인하세요.")
        return # main() 함수 내부에서는 return 사용 가능
    
    # 1.6단계: 상품 메타데이터 컬럼 확인 및 이름 변경 (에러 방지 로직)
    df_oly = verify_and_rename_metadata_columns(df_oly)

    
    # 1. COOS 데이터 클리닝 및 표준화
    df_coos_clean = clean_and_standardize_coos(df_coos)
    
    # 2. 올리브영 성분 목록 분리 및 매핑 키 생성
    print("\n[2단계] 올리브영 성분 목록 분리 및 매핑 키 생성...")
    
    # 성분 목록이 이미 리스트 형태이므로, str.split() 과정 없이 바로 explode 수행
    # .copy()를 사용하여 SettingWithCopyWarning 방지
    df_oly_exploded = df_oly.explode('성분_목록').copy()
    
    # 펼쳐진 성분 컬럼의 값을 '사용_원료명' 컬럼에 복사하고 공백 제거
    # 혹시 모를 타입 문제를 대비해 astype(str) 적용
    df_oly_exploded['사용_원료명'] = df_oly_exploded['성분_목록'].astype(str).str.strip()
    
    # 표준 매핑 키 생성
    df_oly_exploded['표준_매핑_키'] = df_oly_exploded['사용_원료명'].apply(create_standard_name)
    df_oly_exploded.dropna(subset=['표준_매핑_키'], inplace=True)
    
    # 3. 올리브영 성분과 COOS 데이터 통합 (병합)
    print("\n[3단계] 성분 속성 연결을 위한 최종 데이터셋 병합...")
    
    # 병합할 올리브영 컬럼 - (브랜드, 가격, 리뷰수, 평점)은 데이터에 없으므로 제외합니다.
    existing_oly_cols = ['상품ID', '상품명', '카테고리', '사용_원료명', '표준_매핑_키']
    
    # 실제 df_oly_exploded에 존재하는 컬럼만 선택 (상품명만 매핑되었으므로)
    df_oly_slim = df_oly_exploded[[col for col in existing_oly_cols if col in df_oly_exploded.columns]]

    # 표준 매핑 키를 기준으로 COOS 속성 병합
    df_final_normalized = df_oly_slim.merge(
        df_coos_clean, 
        left_on='표준_매핑_키', 
        right_on='표준성분명', 
        how='left'
    )
    
    # 4. 최종 컬럼 정리 및 저장
    # 불필요한 중복 키 제거 (표준성분명 == 표준_매핑_키 이므로 하나만 남김)
    df_final_normalized.drop(columns=['표준성분명', '성분_목록', 'URL'], inplace=True, errors='ignore') 
    
    print(f"✅ 최종 통합 및 정규화된 데이터셋 크기: {len(df_final_normalized)} 행")
    
    df_final_normalized.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')

    print("\n==================================================")
    print(f"✅ 데이터 통합 및 정규화 완료!")
    print(f"결과 파일: {OUTPUT_FILE}")
    print("이 파일은 '상품' - '개별 성분' 단위로 정규화되어 있으며, NameError 문제가 해결되었습니다.")
    print("==================================================")

# ----------------------------------------------------------------------
# --- 메인 실행 블록 ---
# ----------------------------------------------------------------------
if __name__ == "__main__":
    main()



[0.2단계] COOS 데이터 로드 완료: 25750개 성분

[0.1단계] 올리브영 JSON 파일 통합 시작...
✅ 총 2187개 상품 통합 완료.
   ⚠️ 경고: '성분_목록' 컬럼 대신 '성분리스트' 컬럼을 사용하여 이름을 변경했습니다.
   [1.6단계] 메타데이터 컬럼 표준화 시작...
      - '제품명'을(를) '상품명'으로(로) 변경했습니다.
   ✅ 메타데이터 표준화 완료 (상품명만 존재 확인).

[1단계] COOS 성분 데이터 정리 및 표준화 시작...

[2단계] 올리브영 성분 목록 분리 및 매핑 키 생성...

[3단계] 성분 속성 연결을 위한 최종 데이터셋 병합...
✅ 최종 통합 및 정규화된 데이터셋 크기: 90507 행

✅ 데이터 통합 및 정규화 완료!
결과 파일: integrated_product_ingredient_normalized.csv
이 파일은 '상품' - '개별 성분' 단위로 정규화되어 있으며, NameError 문제가 해결되었습니다.


In [5]:
import pandas as pd
import numpy as np
import re
import os
import warnings
from itertools import combinations 
warnings.filterwarnings('ignore', category=FutureWarning)

# ----------------------------------------------------------------------
# --- 설정 ---
# ----------------------------------------------------------------------
# COOS 데이터베이스 파일 (사용자 제공 파일명)
COOS_FILE = "coos_ingredient_database.csv"

# 올리브영 JSON 파일 목록 (사용자 제공 파일명)
OLIVE_YOUNG_FILES = [
    "oliveyoung_로션_raw_limited.json",
    "oliveyoung_미스트_오일_raw_limited.json",
    "oliveyoung_스킨_토너_raw_limited.json",
    "oliveyoung_에센스_세럼_앰플_raw_limited.json",
    "oliveyoung_크림_raw_limited.json",
]
OUTPUT_FILE = "integrated_product_ingredient_normalized.csv"

# ----------------------------------------------------------------------
# --- 데이터 클리닝 유틸리티 (Normalization Functions) ---
# ----------------------------------------------------------------------

def clean_tags(tags):
    """COOS 데이터의 태그 목록에서 불필요한 노이즈를 제거합니다."""
    if not isinstance(tags, str): return ""
    tags_list = [t.strip() for t in tags.split(',')]
    # EWG-Green 태그는 남기고, EWG 등 불필요한 패턴 제거
    NOISE_PATTERNS = re.compile(r'^(AI|KO|EN|JP|EU|식|가능|불가|EWG|EWG-Green)$', re.IGNORECASE)
    cleaned_tags = [t for t in tags_list if t and len(t) > 2 and not NOISE_PATTERNS.match(t)]
    return ', '.join(cleaned_tags)

def create_standard_name(name):
    """성분명에서 특수문자 등을 제거하고 소문자화하여 표준 매핑 키(Standard Key)를 만듭니다."""
    if not isinstance(name, str): return None
    # 괄호와 그 안의 내용 제거 (예: 정제수(달팽이점액여과물) -> 정제수)
    name = re.sub(r'\(.*?\)', '', name).strip()
    # 문자, 숫자, 한글, 영어 외의 모든 문자를 공백으로 치환 후 제거
    cleaned_name = re.sub(r'[^\w가-힣]+', '', name).lower()
    return cleaned_name

def parse_brand_and_name(full_name):
    """
    상품명에서 불필요한 기획/증정 정보를 제거하고 브랜드와 제품명을 분리합니다.
    """
    if not isinstance(full_name, str):
        return None, None
    
    # 1. []나 () 안에 있는 노이즈 제거 (예: [1등올인원], (+100ml증정))
    cleaned_name = re.sub(r'\[.*?\]|\(.*?\)', '', full_name).strip()
    
    # 2. '기획세트', '세트', '증정', '대용량', '본품' 등 상품의 특성 관련 단어 제거
    noise_words = ['기획세트', '기획', '세트', '증정', '대용량', '본품', '단품']
    for word in noise_words:
        cleaned_name = cleaned_name.replace(word, ' ').strip()
    
    # 여러 개의 공백을 하나로 줄임
    cleaned_name = re.sub(r'\s+', ' ', cleaned_name).strip()
    
    # 3. 브랜드명 (첫 번째 단어) 추출 및 제품명 (나머지 부분) 추출
    parts = cleaned_name.split(maxsplit=1)
    brand = parts[0] if parts else None
    product_name = parts[1].strip() if len(parts) > 1 else None
    
    return brand, product_name

def preprocess_mushed_ingredients(full_string):
    """
    붙어버린 성분 경계를 복구하고, 키트 마커를 정리합니다.
    (예: '토코페롤정제수' -> '토코페롤,정제수')
    (예: '시카크림100ml+15ml정제수' -> '시카크림100ml+15ml,정제수')
    """
    # 1. 흔한 시작 성분 그룹: 이 성분들이 이전 성분이나 제품명에 붙어있을 확률이 높음
    START_INGREDIENT_GROUP = r'(정제수|글리세린|다이카프릴릴|나이아신아마이드|메틸프로판다이올|부틸렌글라이콜|프로판다이올|펜틸렌글라이콜|스쿠알란|판테놀|토코페롤|아데노신|시트릭애씨드|소듐클로라이드|세테아릴|시트릭애씨드)'
    
    # 2. 정규식 패턴: (\w) (한글,영문,숫자) 뒤에 바로 (흔한 시작 성분)이 붙어있는 경우
    # 제품명/용량 정보('100ml') 뒤에 성분이 붙어있는 경우도 처리하기 위해 \w 사용
    regex = re.compile(r'(\w)' + START_INGREDIENT_GROUP, re.UNICODE)
    
    # 3. 찾은 패턴을 '성분1,성분2' 형태로 대체하여 쉼표를 삽입합니다.
    processed_string = regex.sub(r'\1,\2', full_string)
    
    # 4. 키트 제품명 마커 정리: '제품명:정제수' 형태에서 콜론 뒤에 쉼표가 없으면 추가하여 분리하기 쉽게 만듭니다.
    processed_string = re.sub(r'([가-힣\s]+?)\s*:\s*([가-힣])', r'\1:\2', processed_string)
    
    # 5. 불필요한 공백/개행 문자 제거 및 정리
    processed_string = re.sub(r'\s{2,}', ' ', processed_string).replace('\n', ' ').strip()
    
    return processed_string

def split_kit_string(ingredient_list_or_str):
    """
    제품명:성분리스트가 합쳐진 문자열을 파싱하여 개별 제품 레코드를 반환합니다.
    (콜론, 대괄호 기반 마커 분리 로직 사용)
    """
    if isinstance(ingredient_list_or_str, list):
        full_string = " ".join([str(item) for item in ingredient_list_or_str]).strip()
    elif isinstance(ingredient_list_or_str, str):
        full_string = ingredient_list_or_str.strip()
    else:
        return []

    if not full_string:
        return []
        
    # 1. **전처리:** 붙어버린 성분 경계를 복구합니다.
    full_string = preprocess_mushed_ingredients(full_string)
    
    # 2. 키트 마커 패턴: '제품명:' (콜론) 또는 '[제품명]' (대괄호)
    KIT_MARKER_PATTERN = r'([가-힣\s]+?\s*:)|(\[.*?\])'
    
    # 마커를 기준으로 분할하고, 마커 자체를 포함하도록 정규식 그룹을 사용
    parts = re.split(f'({KIT_MARKER_PATTERN})', full_string)
    
    products = []
    current_product_name = None
    current_ingredients_str = ""
    
    # 분할된 문자열 처리
    for part in parts:
        if not part or part.isspace():
            continue
            
        # 3. 제품명 마커 확인
        
        # Case 1: '제품명:' 형태의 마커 (예: '올인원 스틱:')
        if part.strip().endswith(':'):
            # 이전 제품의 성분을 처리하고 새 제품 시작
            if current_ingredients_str and current_product_name:
                ingredients = [ing.strip() for ing in current_ingredients_str.split(',') if ing.strip()]
                products.append({'name': current_product_name, 'ingredients': ingredients})
                current_ingredients_str = ""
            
            current_product_name = part.strip().replace(':', '').strip()
            
        # Case 2: '[제품명]' 형태의 마커
        elif part.startswith('[') and part.endswith(']'):
            # 이전 제품의 성분을 처리하고 새 제품 시작
            if current_ingredients_str and current_product_name:
                ingredients = [ing.strip() for ing in current_ingredients_str.split(',') if ing.strip()]
                products.append({'name': current_product_name, 'ingredients': ingredients})
                current_ingredients_str = ""
            
            current_product_name = part.strip().strip('[]').strip()
            
        # Case 3: 성분 문자열 (쉼표가 이미 전처리 단계에서 삽입됨)
        else:
            current_ingredients_str += part
            
    # 4. 마지막 제품의 성분 처리
    if current_ingredients_str:
        ingredients = [ing.strip() for ing in current_ingredients_str.split(',') if ing.strip()]
        
        # 성분 문자열에서 불필요한 키트 잔여물 제거 및 첫 성분 정리
        START_INGREDIENT_GROUP_CLEAN = r'(정제수|글리세린|다이카프릴릴|나이아신아마이드|메틸프로판다이올|부틸렌글라이콜|프로판다이올|펜틸렌글라이콜|스쿠알란|판테놀|토코페롤|아데노신|시트릭애씨드|소듐클로라이드|세테아릴|시트릭애씨드)'
        
        if ingredients and ingredients[0] and re.match(START_INGREDIENT_GROUP_CLEAN, ingredients[0], re.IGNORECASE):
            first_ing = ingredients[0]
            match = re.search(START_INGREDIENT_GROUP_CLEAN, first_ing, re.IGNORECASE)
            if match and match.start() > 0:
                ingredients[0] = first_ing[match.start():].strip()

        
        if not products and not current_product_name:
             # 키트 마커 없이 단일 제품으로 인식
             products.append({'name': None, 'ingredients': ingredients})
        elif current_product_name:
             # 마커가 있었던 마지막 제품
             products.append({'name': current_product_name, 'ingredients': ingredients})
        else:
            # 마커는 없는데 데이터가 남은 경우 (단일 제품으로 강제 처리)
            products.append({'name': None, 'ingredients': ingredients})
            
    return products

# ----------------------------------------------------------------------
# --- 핵심 정규화 로직 함수 (누락된 부분) ---
# ----------------------------------------------------------------------

def split_and_normalize_kit(df_oly):
    """
    올리브영 데이터프레임을 받아, 키트 제품을 개별 상품으로 분리하고,
    각 성분을 행으로 폭발(Explode)시켜 최종 정규화된 데이터프레임을 반환합니다.
    """
    print("\n[2단계] 키트 분리, 성분 폭발, 표준화 시작...")
    
    all_normalized_products = []
    
    for index, row in df_oly.iterrows():
        # 1. 키트 분리 및 성분 추출
        kit_products = split_kit_string(row['성분_목록'])
        
        # 2. 브랜드명 (원래 상품명 기준) 및 제품명(키트 분리 결과) 파싱
        original_product_name = row['상품명']
        original_brand, parsed_product_name = parse_brand_and_name(original_product_name)
        
        # 키트 분리 결과에 따라 반복 처리
        for kit_item in kit_products:
            # 최종 제품명 결정: 키트 분리 시 이름이 있으면 사용, 없으면 원본에서 파싱된 제품명 사용
            final_product_name = kit_item['name'] if kit_item['name'] else parsed_product_name
            
            # 3. 개별 성분 폭발 및 매핑 키 생성
            for ingredient in kit_item['ingredients']:
                if ingredient:
                    # 성분 데이터 레코드 생성
                    record = {
                        '상품ID': row.get('상품ID'),
                        '상품명': original_product_name, # 원본 상품명 유지
                        '브랜드명': original_brand,
                        '제품명': final_product_name, # 정규화된 개별 제품명
                        '카테고리': row.get('카테고리'),
                        'URL': row.get('URL'),
                        '사용_원료명': ingredient, # 올리브영 성분명
                        '표준_매핑_키': create_standard_name(ingredient) # COOS 매핑을 위한 표준 키
                    }
                    all_normalized_products.append(record)

    df_normalized = pd.DataFrame(all_normalized_products)
    print(f"   ✅ 키트 분리 및 성분 폭발 완료. 총 {len(df_normalized)}개의 성분 레코드 생성.")
    return df_normalized


# ----------------------------------------------------------------------
# --- 데이터 로딩 및 통합 함수 ---
# ----------------------------------------------------------------------

def load_and_consolidate_oliveyoung_data(file_list):
    """여러 올리브영 JSON 파일을 통합하고 카테고리 정보를 추가합니다."""
    all_dfs = []
    print("\n[0.1단계] 올리브영 JSON 파일 통합 시작...")
    for file_path in file_list:
        try:
            base_name = os.path.basename(file_path)
            # 파일명에서 카테고리 추출
            category_part = base_name.replace('oliveyoung_', '').replace('_raw_limited.json', '')
            category_name = category_part.replace('_', '/')

            # encoding='utf-8' 명시
            df = pd.read_json(file_path, encoding='utf-8')
            df['카테고리'] = category_name
            all_dfs.append(df)
            
        except FileNotFoundError:
            print(f"   ❌ 경고: 파일 '{file_path}'을 찾을 수 없습니다. 건너뜁니다.")
        except Exception as e:
            print(f"   ❌ 경고: 파일 '{file_path}' 로드 중 오류 발생: {e}. 건너뜁니다.")
            
    if not all_dfs:
        print("   ❌ 오류: 로드된 올리브영 데이터가 없습니다. 스크립트를 종료합니다.")
        return None
        
    df_consolidated = pd.concat(all_dfs, ignore_index=True)
    if '상품ID' not in df_consolidated.columns:
        df_consolidated['상품ID'] = df_consolidated.index.astype(str)
        
    print(f"✅ 총 {len(df_consolidated)}개 상품 통합 완료.")
    return df_consolidated

def clean_and_standardize_coos(df_coos):
    """COOS 데이터를 클리닝하고 표준 매핑 키를 생성합니다."""
    print("\n[1단계] COOS 성분 데이터 정리 및 표준화 시작...")
    
    # COOS 파일의 실제 컬럼명 확인 및 기본값 설정
    coos_name_col = '원료명'
    coos_tag_col = '태그_목록'
    coos_url_col = '상세_URL'
    
    if coos_name_col not in df_coos.columns:
        coos_name_col = df_coos.columns[0] # 첫 번째 컬럼을 원료명으로 간주

    df_coos['태그_목록_정리'] = df_coos[coos_tag_col].apply(clean_tags)
    df_coos['표준성분명'] = df_coos[coos_name_col].apply(create_standard_name)
    
    # 2. 모델에 필요한 속성 컬럼 정의
    coos_cols = ['표준성분명', coos_name_col, '영문명', 'CAS_No', '설명_요약', '태그_목록_정리', coos_url_col]
    
    # 실제 존재하는 컬럼만 선택
    coos_cols_present = [col for col in coos_cols if col in df_coos.columns]

    df_coos_clean = df_coos[coos_cols_present].rename(columns={
        coos_name_col: 'COOS_원료명', 
        coos_url_col: 'COOS_상세_URL'
        }).drop_duplicates(subset=['표준성분명'], keep='first')
    
    return df_coos_clean

def verify_and_rename_ingredient_column(df, expected_col='성분_목록'):
    """
    df에 필수 성분 컬럼이 있는지 확인하고, 없으면 대안 컬럼을 찾아 이름을 변경합니다.
    """
    if expected_col in df.columns:
        return df

    ALTERNATIVE_NAMES = ['성분리스트', 'ingredients', '전성분', 'Ingredient_List', '성분목록']
    
    found_alt = None
    for alt in ALTERNATIVE_NAMES:
        if alt in df.columns:
            df.rename(columns={alt: expected_col}, inplace=True)
            print(f"   ⚠️ 경고: '{expected_col}' 컬럼 대신 '{alt}' 컬럼을 사용하여 이름을 변경했습니다.")
            found_alt = True
            break
    
    if not found_alt:
        print(f"   ❌ 치명적 오류: 통합된 올리브영 데이터에 필수 컬럼 '{expected_col}' 또는 그 대안이 없습니다.")
        print(f"   현재 컬럼 목록: {df.columns.tolist()}")
        raise KeyError(expected_col) 

    return df

def verify_and_rename_metadata_columns(df):
    """상품 메타데이터 컬럼 중 존재하는 컬럼(상품명)만 확인하고 표준화합니다."""
    REQUIRED_MAPPING = {'상품명': ['제품명', 'name', 'product_name']}
    current_cols = df.columns.tolist()
    
    print("   [1.6단계] 메타데이터 컬럼 표준화 시작...")

    for expected, alternatives in REQUIRED_MAPPING.items():
        if expected not in current_cols:
            for alt in alternatives:
                if alt in current_cols:
                    df.rename(columns={alt: expected}, inplace=True)
                    print(f"      - '{alt}'을(를) '{expected}'으로(로) 변경했습니다.")
                    break
    
    print("   ✅ 메타데이터 표준화 완료 (상품명만 존재 확인).")
    return df


# ----------------------------------------------------------------------
# --- 메인 실행 로직 ---
# ----------------------------------------------------------------------
def main():
    
    # 0. 데이터 로드
    df_coos = None 
    coos_loaded = False
    try:
        # COOS 데이터는 CSV 파일이므로 pd.read_csv 사용
        df_coos = pd.read_csv(COOS_FILE, encoding='utf-8')
        print(f"\n[0.2단계] COOS 데이터 로드 완료: {len(df_coos)}개 성분")
        coos_loaded = True
    except FileNotFoundError:
        print(f"\n❌ 오류: COOS 파일 '{COOS_FILE}'을 찾을 수 없습니다. 성분 속성 병합 없이 올리브영 데이터만 정규화됩니다.")
    except Exception as e:
        print(f"\n❌ 오류: COOS 파일 로드 중 예상치 못한 오류 발생: {e}")


    df_oly = load_and_consolidate_oliveyoung_data(OLIVE_YOUNG_FILES)
    if df_oly is None: return

    # 1.5단계: 필수 성분 컬럼 확인 및 이름 변경 (성분 목록 키 표준화)
    try:
        df_oly = verify_and_rename_ingredient_column(df_oly, expected_col='성분_목록')
    except KeyError:
        print("\n   ⚠️ 데이터 오류로 인해 스크립트를 종료합니다. JSON 파일의 성분 목록 키를 확인하세요.")
        return 
    
    # 1.6단계: 상품 메타데이터 컬럼 확인 및 이름 변경
    df_oly = verify_and_rename_metadata_columns(df_oly)

    
    # 1. COOS 데이터 클리닝 및 표준화
    if coos_loaded:
        df_coos_clean = clean_and_standardize_coos(df_coos)
    else:
        df_coos_clean = pd.DataFrame()
    
    # 2. **업데이트된 핵심 로직:** 개별 상품 분리, 브랜드/제품명 파싱, 성분 폭발 및 매핑 키 생성
    # >>>>> 누락되었던 함수 호출 <<<<<
    df_oly_exploded = split_and_normalize_kit(df_oly)
    
    # 3. 올리브영 성분과 COOS 데이터 통합 (병합)
    print("\n[3단계] 성분 속성 연결을 위한 최종 데이터셋 병합...")
    
    existing_oly_cols = ['상품ID', '상품명', '브랜드명', '제품명', '카테고리', 'URL', '사용_원료명', '표준_매핑_키']
    
    # 최종 결과에 포함할 컬럼 (순서 유지를 위해 리스트 사용)
    # CAS_No는 COOS 데이터에만 있으므로 coos_loaded 여부에 관계없이 포함
    final_columns = [
        '상품ID', '브랜드명', '제품명', '카테고리', '사용_원료명', 
        'COOS_원료명', '영문명', '설명_요약', '태그_목록_정리', 'CAS_No', 'COOS_상세_URL', 'URL'
    ]
    
    # 실제 df_oly_exploded에 존재하는 컬럼만 선택
    df_oly_slim = df_oly_exploded[[col for col in existing_oly_cols if col in df_oly_exploded.columns]]

    # 표준 매핑 키를 기준으로 COOS 속성 병합 (COOS 데이터가 있는 경우에만)
    if coos_loaded:
        df_final_normalized = df_oly_slim.merge(
            df_coos_clean, 
            left_on='표준_매핑_키', 
            right_on='표준성분명', 
            how='left'
        )
        # 불필요한 임시 컬럼 제거
        df_final_normalized.drop(columns=['표준성분명', '표준_매핑_키', '상품명'], inplace=True, errors='ignore') 
        df_final_normalized = df_final_normalized[[col for col in final_columns if col in df_final_normalized.columns]]
    else:
        # COOS 데이터가 없으면 병합 없이 올리브영 데이터만 정리
        df_final_normalized = df_oly_slim.drop(columns=['표준_매핑_키', '상품명'], errors='ignore')
        print("   ⚠️ COOS 파일 누락으로 성분 속성(COOS_원료명, 영문명, CAS_No 등) 없이 올리브영 데이터만 정규화됩니다.")

    print(f"✅ 최종 통합 및 정규화된 데이터셋 크기: {len(df_final_normalized)} 행")
    
    # 최종 CSV 파일 저장
    df_final_normalized.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')

    print("\n==================================================")
    print(f"✅ 데이터 통합 및 정규화 완료!")
    print(f"결과 파일: {OUTPUT_FILE}")
    print("브랜드명, 제품명, 개별 성분(COOS 속성 포함)을 기준으로 데이터가 완벽하게 정규화되었습니다.")
    print("==================================================")

# ----------------------------------------------------------------------
# --- 메인 실행 블록 ---
# ----------------------------------------------------------------------
if __name__ == "__main__":
    main()



[0.2단계] COOS 데이터 로드 완료: 25750개 성분

[0.1단계] 올리브영 JSON 파일 통합 시작...
✅ 총 2187개 상품 통합 완료.
   ⚠️ 경고: '성분_목록' 컬럼 대신 '성분리스트' 컬럼을 사용하여 이름을 변경했습니다.
   [1.6단계] 메타데이터 컬럼 표준화 시작...
      - '제품명'을(를) '상품명'으로(로) 변경했습니다.
   ✅ 메타데이터 표준화 완료 (상품명만 존재 확인).

[1단계] COOS 성분 데이터 정리 및 표준화 시작...

[2단계] 키트 분리, 성분 폭발, 표준화 시작...
   ✅ 키트 분리 및 성분 폭발 완료. 총 4644개의 성분 레코드 생성.

[3단계] 성분 속성 연결을 위한 최종 데이터셋 병합...
✅ 최종 통합 및 정규화된 데이터셋 크기: 4644 행

✅ 데이터 통합 및 정규화 완료!
결과 파일: integrated_product_ingredient_normalized.csv
브랜드명, 제품명, 개별 성분(COOS 속성 포함)을 기준으로 데이터가 완벽하게 정규화되었습니다.


In [6]:
import pandas as pd
import numpy as np
import re
import os
import warnings
from itertools import combinations 
warnings.filterwarnings('ignore', category=FutureWarning)

# ----------------------------------------------------------------------
# --- 설정 ---
# ----------------------------------------------------------------------
# COOS 데이터베이스 파일 (사용자 제공 파일명)
COOS_FILE = "coos_ingredient_database.csv"

# 올리브영 JSON 파일 목록 (사용자 제공 파일명)
OLIVE_YOUNG_FILES = [
    "oliveyoung_로션_raw_limited.json",
    "oliveyoung_미스트_오일_raw_limited.json",
    "oliveyoung_스킨_토너_raw_limited.json",
    "oliveyoung_에센스_세럼_앰플_raw_limited.json",
    "oliveyoung_크림_raw_limited.json",
]
OUTPUT_FILE = "integrated_product_ingredient_normalized_2.csv"

# ----------------------------------------------------------------------
# --- 데이터 클리닝 유틸리티 (Normalization Functions) ---
# ----------------------------------------------------------------------

def clean_tags(tags):
    """COOS 데이터의 태그 목록에서 불필요한 노이즈를 제거합니다."""
    if not isinstance(tags, str): return ""
    tags_list = [t.strip() for t in tags.split(',')]
    # EWG-Green 태그는 남기고, EWG 등 불필요한 패턴 제거
    NOISE_PATTERNS = re.compile(r'^(AI|KO|EN|JP|EU|식|가능|불가|EWG|EWG-Green)$', re.IGNORECASE)
    cleaned_tags = [t for t in tags_list if t and len(t) > 2 and not NOISE_PATTERNS.match(t)]
    return ', '.join(cleaned_tags)

def create_standard_name(name):
    """성분명에서 특수문자 등을 제거하고 소문자화하여 표준 매핑 키(Standard Key)를 만듭니다."""
    if not isinstance(name, str): return None
    # 괄호와 그 안의 내용 제거 (예: 정제수(달팽이점액여과물) -> 정제수)
    name = re.sub(r'\(.*?\)', '', name).strip()
    # 문자, 숫자, 한글, 영어 외의 모든 문자를 공백으로 치환 후 제거
    cleaned_name = re.sub(r'[^\w가-힣]+', '', name).lower()
    return cleaned_name

def parse_brand_and_name(full_name):
    """
    상품명에서 불필요한 기획/증정 정보를 제거하고 브랜드와 제품명을 분리합니다.
    """
    if not isinstance(full_name, str):
        return None, None
    
    # 1. []나 () 안에 있는 노이즈 제거 (예: [1등올인원], (+100ml증정))
    cleaned_name = re.sub(r'\[.*?\]|\(.*?\)', '', full_name).strip()
    
    # 2. '기획세트', '세트', '증정', '대용량', '본품' 등 상품의 특성 관련 단어 제거
    noise_words = ['기획세트', '기획', '세트', '증정', '대용량', '본품', '단품']
    for word in noise_words:
        cleaned_name = cleaned_name.replace(word, ' ').strip()
    
    # 여러 개의 공백을 하나로 줄임
    cleaned_name = re.sub(r'\s+', ' ', cleaned_name).strip()
    
    # 3. 브랜드명 (첫 번째 단어) 추출 및 제품명 (나머지 부분) 추출
    parts = cleaned_name.split(maxsplit=1)
    brand = parts[0] if parts else None
    product_name = parts[1].strip() if len(parts) > 1 else None
    
    return brand, product_name

def preprocess_mushed_ingredients(full_string):
    """
    붙어버린 성분 경계를 복구하고, 키트 마커를 정리합니다.
    (예: '토코페롤정제수' -> '토코페롤,정제수')
    """
    # 1. 흔한 시작 성분 그룹: 이 성분들이 이전 성분이나 제품명에 붙어있을 확률이 높음
    START_INGREDIENT_GROUP = r'(정제수|글리세린|다이카프릴릴|나이아신아마이드|메틸프로판다이올|부틸렌글라이콜|프로판다이올|펜틸렌글라이콜|스쿠알란|판테놀|토코페롤|아데노신|시트릭애씨드|소듐클로라이드|세테아릴|시트릭애씨드)'
    
    # 2. 정규식 패턴: (\w) (한글,영문,숫자) 뒤에 바로 (흔한 시작 성분)이 붙어있는 경우
    regex = re.compile(r'(\w)' + START_INGREDIENT_GROUP, re.UNICODE)
    
    # 3. 찾은 패턴을 '성분1,성분2' 형태로 대체하여 쉼표를 삽입합니다.
    processed_string = regex.sub(r'\1,\2', full_string)
    
    # 4. 키트 제품명 마커 정리: '제품명:정제수' 형태에서 콜론 뒤에 쉼표가 없으면 추가하여 분리하기 쉽게 만듭니다.
    processed_string = re.sub(r'([가-힣\s]+?)\s*:\s*([가-힣])', r'\1,\2', processed_string) # 콜론을 쉼표로 변환

    # 5. 불필요한 공백/개행 문자 제거 및 정리
    processed_string = re.sub(r'\s{2,}', ' ', processed_string).replace('\n', ' ').strip()
    
    # 6. 추가 정제: 쉼표 주변의 불필요한 공백 제거
    processed_string = re.sub(r'\s*,\s*', ',', processed_string)
    
    return processed_string

def split_kit_string(ingredient_list_or_str):
    """
    제품명:성분리스트가 합쳐진 문자열을 파싱하여 개별 제품 레코드를 반환합니다.
    (콜론, 대괄호 기반 마커 분리 로직 사용)
    """
    if isinstance(ingredient_list_or_str, list):
        # ***** CRITICAL FIX START *****
        # 리스트 원소들을 공백 대신 쉼표(,)로 연결해야 개별 성분으로 분리 가능
        full_string = ",".join([str(item).strip() for item in ingredient_list_or_str if str(item).strip()]).strip()
        # ***** CRITICAL FIX END *****
    elif isinstance(ingredient_list_or_str, str):
        full_string = ingredient_list_or_str.strip()
    else:
        return []

    if not full_string:
        return []
        
    # 1. **전처리:** 붙어버린 성분 경계를 복구합니다.
    full_string = preprocess_mushed_ingredients(full_string)
    
    # 2. 키트 마커 패턴: '[제품명]' (대괄호)
    # 콜론은 preprocess_mushed_ingredients에서 쉼표로 변환되었으므로 대괄호만 마커로 사용
    KIT_MARKER_PATTERN = r'(\[.*?\])'
    
    # 마커를 기준으로 분할하고, 마커 자체를 포함하도록 정규식 그룹을 사용
    parts = re.split(KIT_MARKER_PATTERN, full_string)
    
    products = []
    current_product_name = None
    current_ingredients_str = ""
    
    # 분할된 문자열 처리
    for part in parts:
        if not part or part.isspace():
            continue
            
        # 3. 제품명 마커 확인
        
        # Case 1: '[제품명]' 형태의 마커
        if part.startswith('[') and part.endswith(']'):
            # 이전 제품의 성분을 처리하고 새 제품 시작
            if current_ingredients_str and current_product_name:
                # 쉼표를 기준으로 성분 분리
                ingredients = [ing.strip() for ing in current_ingredients_str.split(',') if ing.strip()]
                products.append({'name': current_product_name, 'ingredients': ingredients})
                current_ingredients_str = ""
            
            current_product_name = part.strip().strip('[]').strip()
            
        # Case 2: 성분 문자열 (쉼표로 구분됨)
        else:
            # 성분 덩어리가 공백으로 시작하면 제거 (키트 마커 잔여물 방지)
            current_ingredients_str += part.lstrip()
            
    # 4. 마지막 제품의 성분 처리
    if current_ingredients_str:
        # 쉼표를 기준으로 성분 분리
        ingredients = [ing.strip() for ing in current_ingredients_str.split(',') if ing.strip()]
        
        if not products and not current_product_name:
             # 키트 마커 없이 단일 제품으로 인식
             products.append({'name': None, 'ingredients': ingredients})
        elif current_product_name:
             # 마커가 있었던 마지막 제품
             products.append({'name': current_product_name, 'ingredients': ingredients})
        else:
            # 마커는 없는데 데이터가 남은 경우 (단일 제품으로 강제 처리)
            products.append({'name': None, 'ingredients': ingredients})
            
    return products

# ----------------------------------------------------------------------
# --- 핵심 정규화 로직 함수 ---
# ----------------------------------------------------------------------

def split_and_normalize_kit(df_oly):
    """
    올리브영 데이터프레임을 받아, 키트 제품을 개별 상품으로 분리하고,
    각 성분을 행으로 폭발(Explode)시켜 최종 정규화된 데이터프레임을 반환합니다.
    """
    print("\n[2단계] 키트 분리, 성분 폭발, 표준화 시작...")
    
    all_normalized_products = []
    
    for index, row in df_oly.iterrows():
        # 1. 키트 분리 및 성분 추출
        kit_products = split_kit_string(row['성분_목록'])
        
        # 2. 브랜드명 (원래 상품명 기준) 및 제품명(키트 분리 결과) 파싱
        original_product_name = row['상품명']
        original_brand, parsed_product_name = parse_brand_and_name(original_product_name)
        
        # 키트 분리 결과에 따라 반복 처리
        for kit_item in kit_products:
            # 최종 제품명 결정: 키트 분리 시 이름이 있으면 사용, 없으면 원본에서 파싱된 제품명 사용
            final_product_name = kit_item['name'] if kit_item['name'] else parsed_product_name
            
            # 3. 개별 성분 폭발 및 매핑 키 생성
            for ingredient in kit_item['ingredients']:
                if ingredient:
                    # 성분 데이터 레코드 생성
                    record = {
                        '상품ID': row.get('상품ID'),
                        '상품명': original_product_name, # 원본 상품명 유지
                        '브랜드명': original_brand,
                        '제품명': final_product_name, # 정규화된 개별 제품명
                        '카테고리': row.get('카테고리'),
                        'URL': row.get('URL'),
                        '사용_원료명': ingredient, # 올리브영 성분명 (개별 성분)
                        '표준_매핑_키': create_standard_name(ingredient) # COOS 매핑을 위한 표준 키
                    }
                    all_normalized_products.append(record)

    df_normalized = pd.DataFrame(all_normalized_products)
    print(f"   ✅ 키트 분리 및 성분 폭발 완료. 총 {len(df_normalized)}개의 성분 레코드 생성.")
    return df_normalized


# ----------------------------------------------------------------------
# --- 데이터 로딩 및 통합 함수 ---
# ----------------------------------------------------------------------

def load_and_consolidate_oliveyoung_data(file_list):
    """여러 올리브영 JSON 파일을 통합하고 카테고리 정보를 추가합니다."""
    all_dfs = []
    print("\n[0.1단계] 올리브영 JSON 파일 통합 시작...")
    for file_path in file_list:
        try:
            base_name = os.path.basename(file_path)
            # 파일명에서 카테고리 추출
            category_part = base_name.replace('oliveyoung_', '').replace('_raw_limited.json', '')
            category_name = category_part.replace('_', '/')

            # encoding='utf-8' 명시
            df = pd.read_json(file_path, encoding='utf-8')
            df['카테고리'] = category_name
            all_dfs.append(df)
            
        except FileNotFoundError:
            print(f"   ❌ 경고: 파일 '{file_path}'을 찾을 수 없습니다. 건너뜁니다.")
        except Exception as e:
            print(f"   ❌ 경고: 파일 '{file_path}' 로드 중 오류 발생: {e}. 건너뜁니다.")
            
    if not all_dfs:
        print("   ❌ 오류: 로드된 올리브영 데이터가 없습니다. 스크립트를 종료합니다.")
        return None
        
    df_consolidated = pd.concat(all_dfs, ignore_index=True)
    if '상품ID' not in df_consolidated.columns:
        df_consolidated['상품ID'] = df_consolidated.index.astype(str)
        
    print(f"✅ 총 {len(df_consolidated)}개 상품 통합 완료.")
    return df_consolidated

def clean_and_standardize_coos(df_coos):
    """COOS 데이터를 클리닝하고 표준 매핑 키를 생성합니다."""
    print("\n[1단계] COOS 성분 데이터 정리 및 표준화 시작...")
    
    # COOS 파일의 실제 컬럼명 확인 및 기본값 설정
    coos_name_col = '원료명'
    coos_tag_col = '태그_목록'
    coos_url_col = '상세_URL'
    
    if coos_name_col not in df_coos.columns:
        coos_name_col = df_coos.columns[0] # 첫 번째 컬럼을 원료명으로 간주

    df_coos['태그_목록_정리'] = df_coos[coos_tag_col].apply(clean_tags)
    df_coos['표준성분명'] = df_coos[coos_name_col].apply(create_standard_name)
    
    # 2. 모델에 필요한 속성 컬럼 정의
    coos_cols = ['표준성분명', coos_name_col, '영문명', 'CAS_No', '설명_요약', '태그_목록_정리', coos_url_col]
    
    # 실제 존재하는 컬럼만 선택
    coos_cols_present = [col for col in coos_cols if col in df_coos.columns]

    df_coos_clean = df_coos[coos_cols_present].rename(columns={
        coos_name_col: 'COOS_원료명', 
        coos_url_col: 'COOS_상세_URL'
        }).drop_duplicates(subset=['표준성분명'], keep='first')
    
    return df_coos_clean

def verify_and_rename_ingredient_column(df, expected_col='성분_목록'):
    """
    df에 필수 성분 컬럼이 있는지 확인하고, 없으면 대안 컬럼을 찾아 이름을 변경합니다.
    """
    if expected_col in df.columns:
        return df

    ALTERNATIVE_NAMES = ['성분리스트', 'ingredients', '전성분', 'Ingredient_List', '성분목록']
    
    found_alt = None
    for alt in ALTERNATIVE_NAMES:
        if alt in df.columns:
            df.rename(columns={alt: expected_col}, inplace=True)
            print(f"   ⚠️ 경고: '{expected_col}' 컬럼 대신 '{alt}' 컬럼을 사용하여 이름을 변경했습니다.")
            found_alt = True
            break
    
    if not found_alt:
        print(f"   ❌ 치명적 오류: 통합된 올리브영 데이터에 필수 컬럼 '{expected_col}' 또는 그 대안이 없습니다.")
        print(f"   현재 컬럼 목록: {df.columns.tolist()}")
        raise KeyError(expected_col) 

    return df

def verify_and_rename_metadata_columns(df):
    """상품 메타데이터 컬럼 중 존재하는 컬럼(상품명)만 확인하고 표준화합니다."""
    REQUIRED_MAPPING = {'상품명': ['제품명', 'name', 'product_name']}
    current_cols = df.columns.tolist()
    
    print("   [1.6단계] 메타데이터 컬럼 표준화 시작...")

    for expected, alternatives in REQUIRED_MAPPING.items():
        if expected not in current_cols:
            for alt in alternatives:
                if alt in current_cols:
                    df.rename(columns={alt: expected}, inplace=True)
                    print(f"      - '{alt}'을(를) '{expected}'으로(로) 변경했습니다.")
                    break
    
    print("   ✅ 메타데이터 표준화 완료 (상품명만 존재 확인).")
    return df


# ----------------------------------------------------------------------
# --- 메인 실행 로직 ---
# ----------------------------------------------------------------------
def main():
    
    # 0. 데이터 로드
    df_coos = None 
    coos_loaded = False
    try:
        # COOS 데이터는 CSV 파일이므로 pd.read_csv 사용
        df_coos = pd.read_csv(COOS_FILE, encoding='utf-8')
        print(f"\n[0.2단계] COOS 데이터 로드 완료: {len(df_coos)}개 성분")
        coos_loaded = True
    except FileNotFoundError:
        print(f"\n❌ 오류: COOS 파일 '{COOS_FILE}'을 찾을 수 없습니다. 성분 속성 병합 없이 올리브영 데이터만 정규화됩니다.")
    except Exception as e:
        print(f"\n❌ 오류: COOS 파일 로드 중 예상치 못한 오류 발생: {e}")


    df_oly = load_and_consolidate_oliveyoung_data(OLIVE_YOUNG_FILES)
    if df_oly is None: return

    # 1.5단계: 필수 성분 컬럼 확인 및 이름 변경 (성분 목록 키 표준화)
    try:
        df_oly = verify_and_rename_ingredient_column(df_oly, expected_col='성분_목록')
    except KeyError:
        print("\n   ⚠️ 데이터 오류로 인해 스크립트를 종료합니다. JSON 파일의 성분 목록 키를 확인하세요.")
        return 
    
    # 1.6단계: 상품 메타데이터 컬럼 확인 및 이름 변경
    df_oly = verify_and_rename_metadata_columns(df_oly)

    
    # 1. COOS 데이터 클리닝 및 표준화
    if coos_loaded:
        df_coos_clean = clean_and_standardize_coos(df_coos)
    else:
        df_coos_clean = pd.DataFrame()
    
    # 2. **업데이트된 핵심 로직:** 개별 상품 분리, 브랜드/제품명 파싱, 성분 폭발 및 매핑 키 생성
    df_oly_exploded = split_and_normalize_kit(df_oly)
    
    # 3. 올리브영 성분과 COOS 데이터 통합 (병합)
    print("\n[3단계] 성분 속성 연결을 위한 최종 데이터셋 병합...")
    
    existing_oly_cols = ['상품ID', '상품명', '브랜드명', '제품명', '카테고리', 'URL', '사용_원료명', '표준_매핑_키']
    
    # 최종 결과에 포함할 컬럼 (순서 유지를 위해 리스트 사용)
    final_columns = [
        '상품ID', '브랜드명', '제품명', '카테고리', '사용_원료명', 
        'COOS_원료명', '영문명', '설명_요약', '태그_목록_정리', 'CAS_No', 'COOS_상세_URL', 'URL'
    ]
    
    # 실제 df_oly_exploded에 존재하는 컬럼만 선택
    df_oly_slim = df_oly_exploded[[col for col in existing_oly_cols if col in df_oly_exploded.columns]]

    # 표준 매핑 키를 기준으로 COOS 속성 병합 (COOS 데이터가 있는 경우에만)
    if coos_loaded:
        df_final_normalized = df_oly_slim.merge(
            df_coos_clean, 
            left_on='표준_매핑_키', 
            right_on='표준성분명', 
            how='left'
        )
        # 불필요한 임시 컬럼 제거
        df_final_normalized.drop(columns=['표준성분명', '표준_매핑_키', '상품명'], inplace=True, errors='ignore') 
        df_final_normalized = df_final_normalized[[col for col in final_columns if col in df_final_normalized.columns]]
    else:
        # COOS 데이터가 없으면 병합 없이 올리브영 데이터만 정리
        df_final_normalized = df_oly_slim.drop(columns=['표준_매핑_키', '상품명'], errors='ignore')
        print("   ⚠️ COOS 파일 누락으로 성분 속성(COOS_원료명, 영문명, CAS_No 등) 없이 올리브영 데이터만 정규화됩니다.")

    print(f"✅ 최종 통합 및 정규화된 데이터셋 크기: {len(df_final_normalized)} 행")
    
    # 최종 CSV 파일 저장
    df_final_normalized.to_csv(OUTPUT_FILE, index=False, encoding='utf-8-sig')

    print("\n==================================================")
    print(f"✅ 데이터 통합 및 정규화 완료!")
    print(f"결과 파일: {OUTPUT_FILE}")
    print("브랜드명, 제품명, 개별 성분(COOS 속성 포함)을 기준으로 데이터가 완벽하게 정규화되었습니다.")
    print("==================================================")

# ----------------------------------------------------------------------
# --- 메인 실행 블록 ---
# ----------------------------------------------------------------------
if __name__ == "__main__":
    main()



[0.2단계] COOS 데이터 로드 완료: 25750개 성분

[0.1단계] 올리브영 JSON 파일 통합 시작...
✅ 총 2187개 상품 통합 완료.
   ⚠️ 경고: '성분_목록' 컬럼 대신 '성분리스트' 컬럼을 사용하여 이름을 변경했습니다.
   [1.6단계] 메타데이터 컬럼 표준화 시작...
      - '제품명'을(를) '상품명'으로(로) 변경했습니다.
   ✅ 메타데이터 표준화 완료 (상품명만 존재 확인).

[1단계] COOS 성분 데이터 정리 및 표준화 시작...

[2단계] 키트 분리, 성분 폭발, 표준화 시작...
   ✅ 키트 분리 및 성분 폭발 완료. 총 92827개의 성분 레코드 생성.

[3단계] 성분 속성 연결을 위한 최종 데이터셋 병합...
✅ 최종 통합 및 정규화된 데이터셋 크기: 92827 행

✅ 데이터 통합 및 정규화 완료!
결과 파일: integrated_product_ingredient_normalized_2.csv
브랜드명, 제품명, 개별 성분(COOS 속성 포함)을 기준으로 데이터가 완벽하게 정규화되었습니다.
